# DSRT heatmaps

In [1]:
suppressWarnings({suppressPackageStartupMessages({
    library(readxl)
    library(ComplexHeatmap)
    library(ggplot2)
    library(reshape2)
    library(ggpubr)
})})    

Loading data.

In [2]:
dsrt <- as.data.frame(read_xlsx(path = "DSRT.xlsx"))

New names:
• `` -> `...2`
• `` -> `...3`
• `` -> `...4`
• `` -> `...5`
• `` -> `...6`
• `` -> `...7`
• `` -> `...8`
• `` -> `...9`
• `` -> `...10`
• `` -> `...11`


Setting column names.

In [3]:
colnames(dsrt) <- dsrt[1, ]
dsrt <- dsrt[-1, ]

Cleaning the data for the heatmap

In [4]:
# Figure 1
fig1 <- dsrt[dsrt$`Mechanism of action` != "EGFR inhibitor", -c(1, 11)] 
rownames(fig1) <- fig1$DRUG_NAME
fig1 <- fig1[, -1]
fig1 <- t(fig1)
row_names_fig1 <- rownames(fig1)
fig1 <- apply(fig1, 2, as.numeric)
rownames(fig1) <- row_names_fig1

# Figure 6
fig6 <- dsrt[dsrt$`Mechanism of action` == "EGFR inhibitor", -c(1, 11)] 
rownames(fig6) <- fig6$DRUG_NAME
fig6 <- fig6[, -1]
row_names_fig6 <- rownames(fig6)
fig6 <- apply(fig6, 2, as.numeric)
rownames(fig6) <- row_names_fig6

In [5]:
pdf(file = "Heatmap_DSRT_complete_Fig1.pdf", width = 15, height = 7)
options(repr.plot.width = 15, repr.plot.height = 7)
Heatmap(fig1, column_names_rot = 45, column_dend_side = "bottom",
        top_annotation = HeatmapAnnotation(Mechanism = dsrt[dsrt$`Mechanism of action` != "EGFR inhibitor", "Mechanism of action"],
                                        col = list(Mechanism = c("Bcl-2 inhibitor" = "#2f4f4f", 
                                                                "BET inhibitor" = "#a52a2a", 
                                                                "CDK inhibitor" = "#228b22",
                                                                "CHK1 inhibitor" = "#4b0082",
                                                                "HDAC inhibitor" = "#ff8c00",
                                                                "IAPs/SMAC mimetic" = "#d2b48c",
                                                                "Multi-kinase inhibitor" = "#00ff00",
                                                                "PAK inhibitor" = "#00bfff",
                                                                "PARP inhibitor" = "#0000ff",
                                                                "PI3K/mTOR/AKT inhibitor" = "#ff1493",
                                                                "PLK1 inhibitor" = "#ffff54",
                                                                "Ras/Raf/MEK/ERK inhibitor" = "#dda0dd",
                                                                "XPO1/CRM1 inhibitor" = "#7fffd4"))),
        cluster_rows = FALSE,
        column_names_side = "top",
        name = "DSS", 
        border = T, 
        col = c("white", "firebrick"), 
        width = unit(0.5, "cm")*ncol(fig1),
        height = unit(0.5, "cm")*nrow(fig1)
       )
dev.off()

png 
  2

Getting the DSS density.

In [6]:
# Putting all the data in a vector to gather the threshold for sensitivity
all_data_vector <- c() 
for(c in 1:ncol(fig1)){ 
 all_data_vector <- c(all_data_vector, fig1[, c]) 
}

all_data_vector <- na.omit(all_data_vector) 

In [7]:
quantile(x = all_data_vector, probs = 0.75) # 75th percentile is 12.2125

75% 
12.2125

Density plot.

In [8]:
all_data_vector <- as.data.frame(all_data_vector)
colnames(all_data_vector) <- "DSS"
ggsave(ggplot(all_data_vector, aes(x = DSS)) + 
         geom_density() +
         geom_vline(xintercept = 12.2125, 
                    color = "red", 
                    linetype = 2, 
                    size = 0.5) + 
         theme(panel.background = element_blank(),
               axis.line = element_line(color= "black"),
               axis.title = element_text(colour = "black", size = 12),
               axis.text = element_text(colour = "black", size = 12)),
filename = "density_all_data_DSS.pdf", 
width = 5, 
height = 5) 

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”


Now moving on with Figure 6 heatmap, boxplot, and analyses.

In [9]:
pdf(file = "EGFRi_DSRT_Heatmap_Fig6.pdf", width = 10, height = 10)
options(repr.plot.width = 10, repr.plot.height = 10)
Heatmap(fig6, 
        column_names_rot = 45, 
        top_annotation = HeatmapAnnotation(name = "CAV1 protein levels",
                                           CAV1_levels = c("CAV1_low", "CAV1_high", "CAV1_high", "CAV1_low", "CAV1_low",
                                                          "CAV1_high", "CAV1_low", "CAV1_low"), border = TRUE,
                                           col = list(CAV1_levels = c("CAV1_high" = "#c98586",
                                                                     "CAV1_low" = "#e7bbb4"))),
        column_dend_side = "bottom",
        cluster_rows = FALSE,
        column_names_side = "top",
        name = "DSS", 
        border = T, 
        col = c("white", "firebrick"), 
        width = unit(0.5, "cm")*ncol(fig6),
        height = unit(0.5, "cm")*nrow(fig6))
dev.off()

png 
  2

In [10]:
fig6 <- as.data.frame(t(fig6))
fig6$model <- rownames(fig6)
fig6$CAV1_level <- factor(x = c("CAV1_low", 
                                "CAV1_high", 
                                "CAV1_high", 
                                "CAV1_low", 
                                "CAV1_low",
                                "CAV1_high", 
                                "CAV1_low", 
                                "CAV1_low"),
                          levels = c("CAV1_low", "CAV1_high"))
fig6 <- melt(fig6)

Using model, CAV1_level as id variables



In [11]:
options(repr.plot.width = 3, repr.plot.height = 6)
ggsave(plot = ggplot(data = fig6, aes(x = CAV1_level, y = value, fill = CAV1_level)) + 
geom_boxplot()  +
geom_point(aes(fill = CAV1_level), size = 3, shape = 21, position = position_jitterdodge()) +
ylab("DSS") + 
xlab("") + 
scale_fill_manual(values = c("#c98586", "#e7bbb4")) +
stat_compare_means(method = "t.test", comparisons = list(c("CAV1_high", "CAV1_low"))) +
theme(panel.background = element_blank(),
     axis.line = element_line(colour = "black"),
     text = element_text(size = 12, colour = "black"),
     axis.text = element_text(size = 12, colour = "black"),
     legend.position = "bottom",
     legend.key = element_blank()),
filename = "EGFRi_DSS_boxplot_Fig6.pdf", width = 3, height = 6)

[1] FALSE


In [12]:
t.test(value ~ CAV1_level, paired = FALSE,
       data = fig6)


	Welch Two Sample t-test

data:  value by CAV1_level
t = -4.7689, df = 49.81, p-value = 1.653e-05
alternative hypothesis: true difference in means between group CAV1_low and group CAV1_high is not equal to 0
95 percent confidence interval:
 -5.321551 -2.167167
sample estimates:
 mean in group CAV1_low mean in group CAV1_high 
               2.283846                6.028205 


In [13]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 8.8 (Green Obsidian)

Matrix products: default
BLAS/LAPACK: /homedir01/adini22/.conda/envs/cellhashing_analyses/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] ggpubr_0.5.0          reshape2_1.4.4        ggplot2_3.4.2        
[4] ComplexHeatmap_2.14.0 readxl_1.4.1         

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.10         circlize_0.4.15     tidyr_1.2.1        
 [4] png_0.1-8           assertth